# Aurora – Modelling Notebook

## Objectives
- Build and evaluate a machine learning pipeline for Ames Housing Price Prediction.

## Inputs
- Dataset: data/ames.csv

## Outputs
- Trained pipeline (models/)
- Metrics & plots (assets/)
